In [7]:
from training_methods import train_model
from data_gen import *
from argparse import Namespace
from DLGN_enums import *
from sklearn.model_selection import train_test_split
import numpy as np
import torch


In [8]:
use_dataset = True
if use_dataset == False:
    torch.manual_seed(42)

    dim_in = 10
    tree_depth = 3
    num_points = 10000

    X,Y = gen_spherical_data(depth=tree_depth, dim_in=dim_in, type_data='spherical', num_points=num_points, feat_index_start=0,radius=1)


    x_train, x_test, y_train, y_test = train_test_split(
        X, Y, test_size=0.1, random_state=42, stratify=Y
    )

    data_config = {
        'dim_in': dim_in,
        'tree_depth': tree_depth,
        'num_points': num_points,
        'type_data': 'spherical',
        'feat_index_start': 0,
        'radius': 1
    }
    data_config = Namespace(**data_config)
    data = {}
    data['train_data'] = x_train
    data['train_labels'] = y_train 
    data['test_data'] = x_test
    data['test_labels'] = y_test

else:
    DATA_DIR = 'data/dataset2'
    data = {}
    data['train_data'] = torch.tensor(np.load(DATA_DIR + '/x_train.npy'))
    data['train_labels'] = torch.tensor(np.load(DATA_DIR + '/y_train.npy'))
    data['test_data'] = torch.tensor(np.load(DATA_DIR + '/x_test.npy'))
    data['test_labels'] = torch.tensor(np.load(DATA_DIR + '/y_test.npy'))
    data_config = np.load(DATA_DIR + '/config.npy', allow_pickle=True).item()

In [9]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


In [ ]:
config = {
    "device" : device,
    "model_type" : ModelTypes.VN,
    "num_data" : len(data['train_data']),
    "dim_in" : data_config.dim_in,
    "num_hidden_nodes" : [100]*4, 
    "beta" : 30,
    "loss_fn_type" : LossTypes.CE,
    "optimizer_type" : Optim.ADAM,
    "mode" : "pwc",
    "lr_ratio" : 1,
    "log_features" : False,
    "lr" : 0.001,
    "epochs" : 1000,
    "use_wandb" : False
}
config = Namespace(**config)

In [22]:
config = {
    "device" : device,
    "model_type" : ModelTypes.VT,
    "num_data" : len(data['train_data']),
    "dim_in" : data_config.dim_in,
    "num_hidden_nodes" : [10]*4,
    "beta" : 30,
    "mode" : "pwc",
    "value_scale" : 500,
    "BN" : False,
    "prod" : "op",
    "feat" : "sf", 
    "loss_fn_type" : LossTypes.CE,
    "optimizer_type" : Optim.SGD,
    "epochs" : 200,
    "save_freq" : 100,
    "value_freq" : 100,
    "lr": 0.005,
    "vt_fit" : VtFit.NPKSVC,
    "reg" : 1,
    "use_wandb" : False,
}
config = Namespace(**config)

In [ ]:
config = {
    "device" : device,
    "model_type" : ModelTypes.KERNEL,
    "num_data" : len(data['train_data']),
    "dim_in" : data_config.dim_in,
    "width" : 100,
    "depth" : 4,
    "beta" : 30,
    "alpha_init" : None,
    "BN" : True,
    "loss_fn_type" : LossTypes.HINGE,
    "optimizer_type" : Optim.ADAM,
    "train_method" : KernelTrainMethod.PEGASOS,
    "log_features" : False,
    "gates_lr" : 0.001,
    "alpha_lr" : 0.1,
    "reg" : 1,
    "epochs" : 1000,
    "value_freq": 100,
    "num_iter" : 50000,
    'weight_decay' : 0.01,
    'feat' : "cf",
    'threshold' : 0.3,
    "use_wandb" : False,
}
config = Namespace(**config)


In [23]:
model = train_model(data,config)


  0%|          | 0/201 [00:00<?, ?it/s]

Accuracy:  0.50305
Loss before updating alphas at epoch 0  is  0.6944073830604554
Time taken to fit value net:  262.92840051651
Accuracy:  0.87195
Accuracy:  0.64295
Test Accuracy:  0.64295
Loss after updating value_net at epoch 0  is  0.36053292779922486


Loss 0.318192:  50%|████▉     | 100/201 [06:50<02:21,  1.40s/it]  

Accuracy:  0.8851
Loss before updating alphas at epoch 100  is  0.31819229147434236
Time taken to fit value net:  230.14435482025146
Accuracy:  0.89795
Accuracy:  0.65695
Test Accuracy:  0.65695
Loss after updating value_net at epoch 100  is  0.2950963167190552


Loss 0.275219: 100%|█████████▉| 200/201 [12:14<00:00,  1.23it/s]  

Accuracy:  0.9038
Loss before updating alphas at epoch 200  is  0.27521855029463765
